In [1]:
import pysam
from pysam import VariantFile as vcf
import operator
from math import log2
import pandas as pd
from pandas import DataFrame as dataframe
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import scipy
import  os
import os.path
import matplotlib.colors as mcolors
from scipy import stats

In [2]:
def generatedfandcsv(columns,dic,filename):

    row_names=list(dic.keys())
    a=[]
    bs=[]
    #cs=[]
    for x in row_names:
        a.append(x)
        bs.append(dic[x])
        #cs.append(dic[x][1])
    dictforDF=dict()
    dictforDF[columns[0]]=a
    dictforDF[columns[1]]=bs
    #dictforDF[columns[2]]=cs

    df_index=pd.DataFrame(dictforDF)
    df_index.to_csv("../csvfile_start_lengthwithoutX/"+filename+"_start_lenth.csv")
    return df_index


In [3]:
def checkContinuous1(arr):
    res=dict()
    i=0
    while i<len(arr)-1:
        #print("iteration",i)
        if arr[i]==1:
            #temparr=[]
            #res[i]=0
            count=1
            for j in range(i+1,len(arr),1):
                if arr[j]==1:
                    count+=1
                    
                else:
                    
                    break
            #print("count",count)
            res[i]=count
            i+=count
            #print("i",i)
            #temparr.append(count)
        else:
            i+=1
            continue
    return res
            


In [4]:
def getstartAndLength(filepath,filename):
    csv_matrix = np.loadtxt(open(filepath,"rb"),delimiter=",",skiprows=0)
    res=checkContinuous1(csv_matrix)
    df=generatedfandcsv(columns=["start","length"],dic=res,filename=filename)
    return df

In [5]:
for curDir, dirs, files in os.walk(top="../csvfiles_compare_with_0.01pvaluewithoutX/"):
    for file in files:
        filename=file[:file.index(".")]
        filepath=os.path.join(curDir,file)
        getstartAndLength(filepath=filepath,filename=filename)

In [6]:
#读取一个在start_length的文件，然后统计出有少个length和对应的count
def countNumofLength(dic,filepath):
    csv_data = pd.read_csv(filepath, low_memory = False)#防止弹出警告
    csv_df = pd.DataFrame(csv_data)
    arr=np.array(csv_df['length'])
    #print(dic)
    for i in range(len(arr)):
        #print("before iteration",dic)
        if arr[i] in dic:
            #print("in iteration not",dic)
            dic[arr[i]]+=1
        else:
            dic[arr[i]]=1
            #print("in iteration",dic)
    #print(dic)
    return dic

In [7]:
#读取每一个在start_length的文件，然后统计出有少个length和对应的count
mydic=dict()
mhcdic=dict()
for curDir, dirs, files in os.walk(top="../csvfile_start_lengthwithoutX/"):
    for file in files:
            filename=file[:file.index(".")]
            if filename!="mhc_start_lenth":                  
                filepath=os.path.join(curDir,file)
                countNumofLength(mydic,filepath=filepath)
            else:
                 filepath_mhc=os.path.join(curDir,file)
                 countNumofLength(mhcdic,filepath=filepath_mhc)

In [8]:
mhcdic

{1: 7, 3: 2, 2: 4, 6: 1, 8: 1}

In [16]:
mydic

{1: 1832, 2: 359, 3: 77, 4: 16, 5: 10, 6: 2, 7: 2}

In [9]:
def retArrforDrawBarPlot(dic):
    xs=[]
    ys=[]
    sorted_dic=dict()
    for j in sorted(dic.keys()):
        sorted_dic[j]=dic[j]
    for i in sorted_dic.keys():
        xs.append(i)
        ys.append(sorted_dic[i])
    return xs,ys

In [10]:
def drawBarplot(dic,picname):
    fig,ax=plt.subplots(figsize=(10,20))
    x1,y1=retArrforDrawBarPlot(dic)
    ax.bar(x=x1,height=y1)
    ax.set_title(picname)
    for a,b in zip(x1, y1):
        plt.text(a,b,b,ha='center', va='bottom')
    fig.savefig("../graph_Start_lenth/"+picname+".jpg")

In [11]:
def WithThreshold(threshold,dic):
    larger_than_thres=0
    for i in dic.keys():
        if i>=threshold:
            larger_than_thres+=dic[i]
    return larger_than_thres


In [12]:
xAAs_with_threshold=WithThreshold(threshold=6,dic=mydic)

In [17]:
xAAs_with_threshold

4

In [13]:
path = '../csvfiles_start_length/'      # 输入文件夹地址
files = os.listdir(path)   # 读入文件夹
num_xaa= len(files)-1 
print("numxaa",num_xaa)
xAAs_with_threshold_average=xAAs_with_threshold/num_xaa

numxaa 648


In [14]:
xAAs_with_threshold_average

0.006172839506172839

In [15]:
mhc_with_threshold=WithThreshold(threshold=6,dic=mhcdic)
mhc_with_threshold

2